# Excel Processor 開発用ノートブック

このノートブックでは、inputディレクトリのExcelファイルを読み込んで、
プロセッサーの動作を確認しながら開発できます。

In [ ]:
# 必要なライブラリのインポート
import openpyxl
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from openpyxl.utils import get_column_letter
from pathlib import Path
from datetime import datetime
import pandas as pd

# Excel Processorライブラリのインポート
from excel_processor import ExcelProcessor
from excel_processor.base_processor import BaseSheetProcessor
from excel_processor.processors import SummarySheetProcessor, FormatProcessor

## 1. inputディレクトリのファイル一覧

In [ ]:
# inputディレクトリのExcelファイルを一覧表示
input_dir = Path('input')
excel_files = list(input_dir.glob('*.xlsx')) + list(input_dir.glob('*.xls'))
excel_files = [f for f in excel_files if not f.name.startswith('~$')]

print(f"Found {len(excel_files)} Excel file(s):")
for idx, file in enumerate(excel_files, 1):
    print(f"{idx}. {file.name}")

## 2. Excelファイルを読み込む

In [ ]:
# 読み込むファイルを選択（インデックスを変更）
file_index = 0  # 最初のファイル

if excel_files:
    selected_file = excel_files[file_index]
    print(f"Loading: {selected_file.name}")
    
    # openpyxlで読み込み
    wb = openpyxl.load_workbook(selected_file)
    
    print(f"\nSheets: {wb.sheetnames}")
    print(f"Active sheet: {wb.active.title}")
else:
    print("No Excel files found in input directory")

## 3. シートの内容を確認（pandas）

In [ ]:
# pandasで読み込んでプレビュー
if excel_files:
    # 全シートを読み込み
    for sheet_name in wb.sheetnames:
        print(f"\n{'='*60}")
        print(f"Sheet: {sheet_name}")
        print(f"{'='*60}")
        
        df = pd.read_excel(selected_file, sheet_name=sheet_name)
        print(f"Shape: {df.shape}")
        print("\nFirst 5 rows:")
        display(df.head())

## 4. シートの内容を確認（openpyxl）

In [ ]:
# 特定のシートを選択して内容を確認
if excel_files:
    ws = wb.active  # アクティブシートを取得
    # または: ws = wb['シート名']
    
    print(f"Sheet: {ws.title}")
    print(f"Rows: {ws.max_row}, Columns: {ws.max_column}")
    print("\nFirst 10 rows:")
    
    for row in ws.iter_rows(min_row=1, max_row=min(10, ws.max_row), values_only=True):
        print(row)

## 5. カスタムプロセッサーのテスト

ここで独自のプロセッサーを作成してテストできます。

In [ ]:
# カスタムプロセッサーの例
class TestProcessor(BaseSheetProcessor):
    """テスト用プロセッサー"""
    
    def process(self, workbook, file_path):
        self.log("Test processor running...")
        
        # 例: 全シートの情報を出力
        for sheet_name in workbook.sheetnames:
            ws = workbook[sheet_name]
            self.log(f"Sheet '{sheet_name}': {ws.max_row} rows, {ws.max_column} cols")
        
        # 例: 新しいシートを追加
        test_sheet = self.get_or_create_sheet(workbook, "TestSheet")
        test_sheet['A1'] = "This is a test sheet"
        test_sheet['A1'].font = Font(bold=True, size=14)
        
        self.log("Test processor completed")
        return workbook

# プロセッサーをテスト
if excel_files:
    # ワークブックのコピーを作成してテスト
    test_wb = openpyxl.load_workbook(selected_file)
    
    processor = TestProcessor(config={'test_param': 'value'})
    test_wb = processor.process(test_wb, str(selected_file))
    
    print("\nProcessed sheets:")
    print(test_wb.sheetnames)

## 6. 組み込みプロセッサーのテスト

In [ ]:
# SummarySheetProcessorのテスト
if excel_files:
    test_wb = openpyxl.load_workbook(selected_file)
    
    summary_processor = SummarySheetProcessor(config={
        'sheet_name': 'Summary',
        'position': 0
    })
    
    test_wb = summary_processor.process(test_wb, str(selected_file))
    
    print("After SummarySheetProcessor:")
    print(f"Sheets: {test_wb.sheetnames}")
    
    # サマリーシートの内容を確認
    summary_sheet = test_wb['Summary']
    print("\nSummary sheet content:")
    for row in summary_sheet.iter_rows(min_row=1, max_row=min(15, summary_sheet.max_row), values_only=True):
        print(row)

In [ ]:
# FormatProcessorのテスト
if excel_files:
    test_wb = openpyxl.load_workbook(selected_file)
    
    format_processor = FormatProcessor(config={
        'header_color': '4472C4',
        'font_name': 'Arial',
        'font_size': 11,
        'apply_borders': True,
        'auto_width': True
    })
    
    test_wb = format_processor.process(test_wb, str(selected_file))
    
    print("FormatProcessor applied successfully")
    print(f"Sheets: {test_wb.sheetnames}")

## 7. 処理結果をプレビュー保存

In [ ]:
# テスト結果を保存（プレビュー用）
if excel_files:
    preview_dir = Path('output/preview')
    preview_dir.mkdir(parents=True, exist_ok=True)
    
    preview_file = preview_dir / f"preview_{selected_file.name}"
    test_wb.save(preview_file)
    
    print(f"Preview saved to: {preview_file}")
    print("\nOpen this file to check the results!")

## 8. 複数プロセッサーを組み合わせてテスト

In [ ]:
# 複数のプロセッサーを順番に適用
if excel_files:
    test_wb = openpyxl.load_workbook(selected_file)
    
    # プロセッサーのリスト
    processors = [
        SummarySheetProcessor(config={'sheet_name': 'Summary', 'position': 0}),
        FormatProcessor(config={
            'header_color': '70AD47',
            'font_name': 'Arial',
            'font_size': 11,
            'apply_borders': True,
            'auto_width': True,
            'exclude_sheets': ['Summary']
        })
    ]
    
    # 順番に適用
    for processor in processors:
        test_wb = processor.process(test_wb, str(selected_file))
    
    # 保存
    combined_file = preview_dir / f"combined_{selected_file.name}"
    test_wb.save(combined_file)
    
    print(f"Combined processing completed")
    print(f"Saved to: {combined_file}")
    print(f"Sheets: {test_wb.sheetnames}")

## 9. ExcelProcessorの完全テスト

実際のExcelProcessorを使用して処理をテストします。

In [ ]:
# ExcelProcessorを使用した完全テスト
# 注意: これを実行するとinputフォルダのファイルが移動されます

# テスト用にinputファイルをコピーして実行することをお勧めします
# import shutil
# test_input = Path('input_test')
# test_input.mkdir(exist_ok=True)
# for f in excel_files:
#     shutil.copy(f, test_input / f.name)

# processor = ExcelProcessor(
#     input_dir='input_test',
#     output_dir='output_test',
#     processors=[
#         SummarySheetProcessor(config={'sheet_name': 'Summary', 'position': 0}),
#         FormatProcessor(config={'header_color': '4472C4'})
#     ]
# )

# processor.run()

print("Uncomment the code above to run full ExcelProcessor test")

## 10. カスタムプロセッサーの開発テンプレート

In [ ]:
# 新しいプロセッサーを開発する際のテンプレート
class MyNewProcessor(BaseSheetProcessor):
    """
    カスタムプロセッサーのテンプレート
    
    設定例:
        my_setting: "value"
        another_setting: 123
    """
    
    def process(self, workbook, file_path):
        # 設定値を取得
        my_setting = self.config.get('my_setting', 'default_value')
        
        self.log(f"Processing with setting: {my_setting}")
        
        # ここに処理ロジックを実装
        for sheet_name in workbook.sheetnames:
            ws = workbook[sheet_name]
            
            # 例: シートごとの処理
            self.log(f"Processing sheet: {sheet_name}")
            
            # セルの処理例
            for row in ws.iter_rows(min_row=1, max_row=ws.max_row):
                for cell in row:
                    # セルごとの処理
                    pass
        
        self.log("Processing completed")
        return workbook

# テスト
if excel_files:
    test_wb = openpyxl.load_workbook(selected_file)
    my_processor = MyNewProcessor(config={'my_setting': 'test_value'})
    test_wb = my_processor.process(test_wb, str(selected_file))
    print("Custom processor test completed")